In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import string
import re
import os
from nltk.corpus import stopwords
import math
import random
from sklearn.feature_extraction.text import TfidfVectorizer


# Creating a structed table based on a semi-structed text file 

# TESTS START HERE

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [3]:
# Set the file path
file_path = "/home/marcnaweb/code/marcnaweb/car_recommendation_engine/raw_data/text_files/extracted_datasheets/"


In [4]:
files = os.listdir(file_path)
random.shuffle(files) # shuffle the files to get a random sample
text_files = files[1000:1100]  # [ f"{file_path}1.txt", f"{file_path}2.txt", f"{file_path}3.txt",  f"{file_path}4.txt" ,  f"{file_path}5.txt"]

In [5]:
text_content = []

for file in text_files:
    with open(f"{file_path}{file}", 'r') as file:
        content = file.read()
        try:
            content = content.split('Ficha Técnica')[1]
            content = content.split('Fotos')[0]
            #text_content.append(content)
            #text_content.append(content)
            content = content.split('\n')
            #print(content)


        except:
            pass
    temp = []
    for line in content:
        #pass
        # Remove words in between brackets
        line = re.sub(r'\[.*?\]', ' ', line)
        words = line.split()
        # Remove words that contain a special character
        words = [word for word in words if all(  c == '\n' or c == '/'  or c =='.' or c == ',' for c in string.punctuation if c in word)]

        #include a position indicator to the words that are not stopwords
        # from left to right
        words = [f"{i}_{word}" if word not in stopwords.words('portuguese') else word for i, word in enumerate(words)]
        # from right to left
        words.reverse()
        words = [f"{i}_{word}" if word not in stopwords.words('portuguese') else word for i, word in enumerate(words)]
        #reput the words in the right order
        words.reverse()

        cleaned_line = ' '.join(words)
        #print(cleaned_line)
        text_content.append(cleaned_line)

#print(text_content)


In [6]:
words = ["de", '12,3.5', 'bb']


words = [f"{i}_{word}" if word not in stopwords.words('portuguese') else word for i, word in enumerate(words)]

#words.reverse()
#words = [f"{i}_{word}" for i, word in enumerate(words)]
#words.reverse()

words

['de', '1_12,3.5', '2_bb']

In [7]:
enumerate(words)

In [8]:

corpus = text_content
vectorizer = TfidfVectorizer(min_df=1 , stop_words=stopwords.words('portuguese') , token_pattern=r'\b[\w_,.]*[_,.][\w_,.]*\b' )  #
X = vectorizer.fit(corpus)
vectorizer.transform(corpus).toarray()
vectorizer.get_feature_names_out()

weighted_words = pd.DataFrame(vectorizer.fit_transform(corpus).toarray(),
                 columns = vectorizer.get_feature_names_out(), index = corpus)

weighted_words.reset_index(inplace=True)
weighted_words["temp"] = weighted_words["index"].apply(lambda x: len(x.split()))
weighted_words =    weighted_words[weighted_words["temp"] > 0 ].drop(columns=["temp"])
weighted_words.reset_index(inplace=True)
weighted_words.drop(columns=["level_0"], inplace=True)
#weighted_words


In [9]:
#weighted_words[weighted_words.index == 5245]['index'].values[0]

In [10]:

def get_top_words(line_weight, cutting_value):
    print(cutting_value)
    top_words = line_weight[line_weight[line_number] > cutting_value]
    #print(top_words.index.tolist())
    value_to_return = top_words.index.tolist()
    value_to_return.reverse()
    return value_to_return  #we reverse the list to get the less important words first


In [11]:
a = ["a","b", "c"]#.reverse()
a.reverse()
print(a)

['c', 'b', 'a']


In [12]:



def get_label_and_value(line, value_words):
    # Get the label and the value
    #print(line)

    # Create a list of words

    words = value_words
    #print(words)

    # Create a regex pattern to match the string between any of the words, including the words
    #pattern = r'({}).*?'.format('|'.join(words)) * len(words)
    #pattern = r'({}\S*)?'.format('|'.join(words)) * len(words)
    #pattern = r'({}\S*)'.format('|'.join(words))
    #print(pattern)
    #pattern = r'({}[^\s]*)'.format('|'.join(words))  * len(words)
    pattern = r'({}[^\s]*).*?'.format('|'.join(words)) * len(words)


    # Find the string between the words
    match = re.search(pattern, line, re.DOTALL)
    #match = re.findall(pattern, line, re.DOTALL)

    print(match)


    if match:
        # get the value words
        result = match.group()
        #clean the string from position indicators _0, _1, _2, etc
        pattern = r'\b\w*_'
        result = re.sub(pattern, '', result)
        print(f"found value: {result}")

        # remove the value words from the line to get the label
        #label = line
        #for word in match.group().split():
        #    if word not in stopwords.words('portuguese'):
        #        label = label.replace(word, '')
        #        label = label.replace('  ', ' ') # remove double spaces

        #remove the result from line to get the label
        line = re.sub(pattern, '', line)
        label = line.replace(result, '')
        label = label.replace('  ', ' ') # remove double spaces

        #clean the string from position indicators _0, _1, _2, etc
        pattern = r'\b\w*_'
        label = re.sub(pattern, '', label)
        print("found label: " + label)



        return label, result

    else:
        print('No match found')






#print(line)
#get_label_and_value(line.lower(), top_words)

In [13]:
#get_label_and_value("Torque máximo 22,1 kgfm".lower(), [  'máximo', '22' ])
"okok sffsfsf".replace("sf", "")

'okok f'

In [14]:
#test line numbers: 5515, 270, 5245, 562

label_values_list = []

for line_number in weighted_words.index.tolist():
    print(f"line_number: {line_number}")

    line = weighted_words[weighted_words.index == line_number]['index'].values[0]

    if len(line.split()) > 1:
        print(line)

        #get line weight
        line_weight = weighted_words[weighted_words.index == line_number].select_dtypes(include=[np.number]).T.sort_values(line_number, ascending=False)

        #get cutting value =  mean + (standard deviation) divided by (count)
        cutting_value = line_weight[line_weight[line_number] > 0].describe().loc['mean'] + line_weight[line_weight[line_number] > 0].describe().loc['std'] / line_weight[line_weight[line_number] > 0].describe().loc['count']
        cutting_value = cutting_value.values[0]

        top_words = get_top_words(line_weight, cutting_value)
        label_values_list.append(get_label_and_value(line.lower(), top_words))



#print the cutting value
#print(f"Cutting_value: {cutting_value}")
#print(line_weight.head(10))
#print(line_weight[line_weight[line_number] > 0].describe())


line_number: 0
4_0_Nissan 3_1_Quest 2_2_GXE 1_3_3.0 0_4_V6
0.44956884525250684
<re.Match object; span=(0, 36), match='4_0_nissan 3_1_quest 2_2_gxe 1_3_3.0'>
found value: nissan quest gxe 3.0
found label:  v6
line_number: 1
1_0_Ano 0_1_1996
0.7931280059548417
<re.Match object; span=(8, 16), match='0_1_1996'>
found value: 1996
found label: ano 
line_number: 2
1_0_Preço 0_1_18.918
0.7931280059548417
<re.Match object; span=(10, 20), match='0_1_18.918'>
found value: 18.918
found label: preço 
line_number: 3
line_number: 4
1_0_Propulsão 0_1_Combustão
0.7078452643570318
<re.Match object; span=(14, 27), match='0_1_combustão'>
found value: combustão
found label: propulsão 
line_number: 5
1_0_Combustível 0_1_Gasolina
0.7319502672088374
<re.Match object; span=(16, 28), match='0_1_gasolina'>
found value: gasolina
found label: combustível 
line_number: 6
line_number: 7
1_0_Procedência 0_1_Importado
0.7306107351878475
<re.Match object; span=(16, 29), match='0_1_importado'>
found value: importado
fou

In [15]:
label_values_list

[(' v6', 'nissan quest gxe 3.0'),
 ('ano ', '1996'),
 ('preço ', '18.918'),
 ('propulsão ', 'combustão'),
 ('combustível ', 'gasolina'),
 ('procedência ', 'importado'),
 ('garantia ', '1 ano'),
 ('configuração ', 'minivan'),
 ('porte ', 'médio'),
 ('lugares ', '7'),
 ('portas ', '3'),
 ('série ', 'v40'),
 ('índice cnw ', '43,63'),
 ('ranking cnw ', '17414'),
 ('nota do leitor ', '8,5'),
 ('instalação ', 'dianteiro'),
 ('disposição ', 'transversal'),
 ('aspiração ', 'natural'),
 ('alimentação injeção ', 'multiponto'),
 ('cilindros ', '6 em v'),
 ('comando de válvulas no cabeçote', 'único'),
 ('válvulas por cilindro ', '2'),
 ('diâmetro do cilindro mm', '87'),
 ('curso do pistão mm', '83'),
 ('razão de compressão', ''),
 ('cilindrada unitária ', '493'),
 ('deslocamento ', '2960'),
 ('potência máxima cv', '153'),
 ('regime potência máx. rpm', '4800'),
 ('torque máximo kgfm', '24,5'),
 ('regime torque máx. rpm', '4400'),
 ('peso/potência kg/cv', '11,75'),
 ('torque específico kgfm/litro', 

In [16]:
weighted_words[weighted_words.index == 265]['index'].values[0]

'3_0_Elemento 2_1_elástico 1_2_Mola 0_3_helicoidal'

In [24]:
line_number = 40850
line = weighted_words[weighted_words.index == line_number ]['index'].values[0]
line_weight = weighted_words[weighted_words.index == line_number].select_dtypes(include=[np.number]).T.sort_values(line_number, ascending=False)

#get cutting value =  mean + (standard deviation) divided by (count)
cutting_value = line_weight[line_weight[line_number] > 0].describe().loc['mean'] + line_weight[line_weight[line_number] > 0].describe().loc['std'] / line_weight[line_weight[line_number] > 0].describe().loc['count']
cutting_value = cutting_value.values[0]

top_words = get_top_words(line_weight, cutting_value)
print(top_words)
print(line )
get_label_and_value(line.lower(), top_words)

print(line_weight[line_weight[line_number] > 0].describe())
line_weight.head(10)


0.5966766676280092
['1_1_225']
2_0_Dianteiros 1_1_225/60 0_2_R16
<re.Match object; span=(15, 25), match='1_1_225/60'>
found value: 225/60
found label: dianteiros r16
          40850
count  3.000000
mean   0.574875
std    0.065404
min    0.504609
25%    0.545322
50%    0.586036
75%    0.610008
max    0.633981


,40850
1_1_225,0.633981
0_2_r16,0.586036
2_0_dianteiros,0.504609
1_2_465,0.000000
1_2_45,0.000000
"1_2_45,8",0.000000
"1_2_45,9",0.000000
1_2_450,0.000000
1_2_4500,0.000000
1_2_454,0.000000


In [18]:
import re
pattern = r"(?u)\b\w\w+\b|\b\d{1,2}[.,]?\d*\b|\b\d\b" #old

pattern = r'\b[a-zA-Z0-9.,]{1,}\b|\b\d\b'

pattern = r"(?u)\b\w\w+\b|\b\d{1,2}[.,]?\d*\b|\b\d\b"

pattern = r'\b[a-zA-Z]{2,}\b|\b\d\b|\b\d{1,2}[.,]?\d*\b'

pattern = r"(?<=\s|^)[a-zA-Z]{2,}(?=\s|$)|(?<=\s|^)\d(?=\s|$)|(?<=\s|^)\d{1,2}[.,]?\d*(?=\s|$)"

pattern = r'\b[a-zA-Z]{2,}\b|\b\d\b|\b\d{1,2}[.,]\d*\b'

pattern = r'\b\w*_\w*\b'

pattern = r'\b[\w_,.]*[_,.][\w_,.]*\b'

pattern = r'\b[\w_,.]*[_,.][\w_,.]*\b'   #used by tdidf vectorizer

text = "0_Torque 1_máximo 2_16,5 3_kgfm 4_a 5_4.500 6_rpm 7_em 8_4 9_cilindros"

text = "2_0_Área 1_1_frontal 0_2_3 "
matches = re.findall(pattern, text)

print(matches)  # Output: ['12', 'with', 'and', '3']


['2_0_Área', '1_1_frontal', '0_2_3']


In [19]:
import re

text = "This is a_test_string but not this"
pattern = r'\b\w*_'
substituted_text = re.sub(pattern, '', text)

print(substituted_text)  # Output: 'This is string but not this'

This is string but not this


In [20]:
import re

text = "0_Torque 1_máximo 2_16,5 3_kgfm 4_a 5_4.500 6_rpm 7_em 8_4 9_cilindros"
pattern = r'\b\w*_'
substituted_text = re.sub(pattern, '', text)

print(substituted_text)  # Output: 'This is string but not this'

Torque máximo 16,5 kgfm a 4.500 rpm em 4 cilindros


In [21]:
import re

# Create a sample text
text = 'equipamento de série capô sustentado por mola a...'.lower()

# Create a list of words
words = top_words

# Create a regex pattern to match the string between any of the words, including the words
pattern = r'({}).*?'.format('|'.join(words)) * len(words)

print(pattern)

# Find the string between the words
match = re.search(pattern, text, re.DOTALL)
print(match)

if match:
    result = match.group()
    print(result)
else:
    print('No match found')


<re.Match object; span=(0, 0), match=''>

